In [1]:
# load data
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras import regularizers
import random

# get list of hold out data for ensemble training
random.seed(17332)
nums = random.sample(range(0,1507),60)


training_path = "../../Test_data/train.json"
testing_path = "../../Test_data/test.json"

train_data_whole = pd.read_json(training_path)
train_data = train_data_whole.select(lambda x: x not in nums)

test_data = pd.read_json(testing_path)


# no third for inc angle
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))    # , c)))

    return np.array(imgs)

X_orig = get_scaled_imgs(train_data)

# resize to 150x150
X = []
for i in X_orig:
    X.append(cv2.resize(i, (150,150)))
X = np.array(X)

Y = train_data["is_iceberg"]


X_full = X
Y_full = Y


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, 
                                                                         random_state=42)

print("size x_train", len(X_train))
print("size x_test", len(X_test))
print("size y_train", len(y_train))
print("size y_test", len(y_test))


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement


size x_train 1158
size x_test 386
size y_train 1158
size y_test 386


In [4]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(150, 150, 3))  

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, kernel_regularizer=regularizers.l2(.1),
          bias_regularizer=regularizers.l2(.1))(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

print("compiled")


compiled


In [5]:
# train the model on the new data for a few epochs, just to get the top layer going

# training code
#%% Image data augmentation 
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,               # set input mean to 0 over the dataset
    samplewise_center=False,                # set each sample mean to 0
    featurewise_std_normalization=False,    # divide inputs by std of the dataset
    samplewise_std_normalization=False,     # divide each input by its std
    zca_whitening=False,                    # apply ZCA whitening
    rotation_range=10,                      # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,                  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,                 # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,                   # randomly flip images
    vertical_flip=True)                     # randomly flip images

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_full, Y_full, shuffle=True,
                    batch_size=32), epochs = 30,
                     verbose= 1) #, validation_data = (X_test, y_test))   # validation_data = (X_test, y_test)

Epoch 1/30
49/49 [==============================] - 10s 194ms/step - loss: 135.5199 - acc: 0.5256
Epoch 2/30
49/49 [==============================] - 6s 129ms/step - loss: 131.6796 - acc: 0.5403
Epoch 3/30
49/49 [==============================] - 6s 131ms/step - loss: 127.9304 - acc: 0.5650
Epoch 4/30
49/49 [==============================] - 6s 132ms/step - loss: 124.2802 - acc: 0.5651
Epoch 5/30
49/49 [==============================] - 6s 131ms/step - loss: 120.7237 - acc: 0.5925
Epoch 6/30
49/49 [==============================] - 6s 131ms/step - loss: 117.2488 - acc: 0.6129
Epoch 7/30
49/49 [==============================] - 6s 130ms/step - loss: 113.8703 - acc: 0.6116
Epoch 8/30
49/49 [==============================] - 6s 131ms/step - loss: 110.5543 - acc: 0.6506
Epoch 9/30
49/49 [==============================] - 6s 129ms/step - loss: 107.3382 - acc: 0.6327
Epoch 10/30
49/49 [==============================] - 6s 131ms/step - loss: 104.2237 - acc: 0.6530
Epoch 11/30
49/49 [=========

In [ ]:

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)






In [6]:

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
#for layer in model.layers[:249]:
#   layer.trainable = False
#for layer in model.layers[249:]:
#   layer.trainable = True
# training them all
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy',
              metrics=['accuracy'])

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "model_weights_transf_incV3_ensem.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=15)

print("recompiled")


recompiled


In [7]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
# train the model on the new data for a few epochs, just to get the top layer going

# training code
#%% Image data augmentation 
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,               # set input mean to 0 over the dataset
    samplewise_center=False,                # set each sample mean to 0
    featurewise_std_normalization=False,    # divide inputs by std of the dataset
    samplewise_std_normalization=False,     # divide each input by its std
    zca_whitening=False,                    # apply ZCA whitening
    rotation_range=20,                      # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,                  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,                 # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,                   # randomly flip images
    vertical_flip=True)                     # randomly flip images

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_full, Y_full, shuffle=True,
                    batch_size=32), epochs = 85,
                     verbose= 1) #,callbacks = callbacks) # validation_data = (X_test, y_test),
                   

Epoch 1/85
49/49 [==============================] - 32s 643ms/step - loss: 53.9636 - acc: 0.6767
Epoch 2/85
49/49 [==============================] - 23s 466ms/step - loss: 51.8268 - acc: 0.7385
Epoch 3/85
49/49 [==============================] - 23s 467ms/step - loss: 49.7979 - acc: 0.7545
Epoch 4/85
49/49 [==============================] - 23s 467ms/step - loss: 47.8302 - acc: 0.7667
Epoch 5/85
49/49 [==============================] - 23s 467ms/step - loss: 45.9096 - acc: 0.7947
Epoch 6/85
49/49 [==============================] - 23s 467ms/step - loss: 44.0469 - acc: 0.8067
Epoch 7/85
49/49 [==============================] - 23s 467ms/step - loss: 42.2777 - acc: 0.8010
Epoch 8/85
49/49 [==============================] - 23s 467ms/step - loss: 40.5399 - acc: 0.8164
Epoch 9/85
49/49 [==============================] - 23s 467ms/step - loss: 38.8710 - acc: 0.8195
Epoch 10/85
49/49 [==============================] - 23s 467ms/step - loss: 37.2520 - acc: 0.8242
Epoch 11/85
49/49 [==========

In [8]:
# if doing no val set (i.e. final) only
filepath_full = 'model_weights_transfer_ensem.hdf5'
model.save(filepath_full)
print("complete")

complete


In [3]:
# from above, need to see the amount of epochs (80 +) that val stops increasing at, then do 
# that in one full go and save the model
filepath_full = 'model_weights_transfer_ensem.hdf5'
from keras.models import load_model
import pandas as pd
# if not using val
file_path_use = filepath_full
# if using val
#file_path_use = file_path
inf_model = load_model(file_path_use)

#score = inf_model.evaluate(X_test, y_test, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

# for hold out submission to train ensemble
hold_out_train = train_data_whole.iloc[nums]


X_orig_test = get_scaled_imgs(hold_out_train)

# resize to 150x150
X_sub = []
for i in X_orig_test:
    X_sub.append(cv2.resize(i, (150,150)))
X_sub = np.array(X_sub)


predicted_test=inf_model.predict(X_sub)

print("len of pred test", len(predicted_test))
print("len of id", len(hold_out_train['id']))

submission = pd.DataFrame()
submission['id']=hold_out_train['id']
submission['is_iceberg']=predicted_test
submission.to_csv('sub_full_transfer_incV3_ensem.csv', index=False)

len of pred test 60
len of id 60
